# K-Nearest Neighbours (KNN) from scratch!

Given:
    
    X: a numpy array of shape mxn, m being the number of rows and n being the number of features
    Y: Y of shape mx1 (which is not needed for PCA

In [1]:
from sklearn.datasets import load_breast_cancer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the breast cancer dataset
cancer = load_breast_cancer()
X_cancer = cancer['data']
y_cancer = cancer['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_cancer, y_cancer, test_size=0.2, random_state=42)

X_train_mu = np.mean(X_train, axis=0)
X_train_std = np.std(X_train, axis=0)
X_train_normalized = (X_train - X_train_mu) / X_train_std
X_test_normalized = (X_test - X_train_mu ) / X_train_std

### First, we have to calculate the distance between the data point and each data point in our training corpus.
### There are multiple distance measures:

#### Euclidean:
$$
d(p,q) = \sqrt{\sum_{i=1}^{n}{(q_i - p_i)^2}}
$$

In [2]:
def euclidean_0(p, q):
    # p and q are both of shape 1, n
    total = 0
    for i in range(len(p)):
        total += (p[i] - q[i]) ** 2
    return total

def euclidean_1(p, q):
    # p and q are both of shape 1, n
    p, q = np.array(p), np.array(q)
    return np.sqrt(np.sum((p - q) ** 2))

def euclidean_2(X, q):
    # X is of shape m, n and q is of shape 1, n
    if X.ndim > 2:
        raise ValueError("X has more than 2 dimensions.")
    q = q.reshape(1, -1)
    m, n = X.shape
    Q = np.repeat(q, m, axis=0) # now has shape m, n; works even without this line
    dist = np.sqrt(np.sum((X - Q) ** 2, axis=1))
    return dist.reshape(-1, 1) # returns shape m, 1

def euclidean_3(X, q):
    """
    Compute the Euclidean distance between each row of a 2-D array and a 1-D point.
    
    Parameters:
    X : 2-D array of shape (m, n)
    q : 1-D array of shape (n,)
    
    Returns:
    dist : 1-D array of shape (m,)
        The Euclidean distance between each row of X and q.
    """
    if X.ndim > 2:
        raise ValueError("X has more than 2 dimensions.")
    return np.sqrt(np.sum((X - q) ** 2, axis=1))

In [3]:
# Define test data
p = np.random.rand(50)
q = np.random.rand(50)
X = np.random.rand(1000, 50)

# Test euclidean_0
print("Testing euclidean_0...")
%timeit -n 100 [euclidean_0(x, q) for x in X]

# Test euclidean_1
print("\nTesting euclidean_1...")
%timeit -n 100 [euclidean_1(x, q) for x in X]

# Test euclidean_2
print("\nTesting euclidean_2...")
%timeit -n 100 euclidean_2(X, q)

# Test euclidean_3
print("\nTesting euclidean_3...")
%timeit -n 100 euclidean_3(X, q)

Testing euclidean_0...
23.7 ms ± 548 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Testing euclidean_1...
8.85 ms ± 840 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Testing euclidean_2...
110 µs ± 13.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Testing euclidean_3...
106 µs ± 8.74 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Manhattan:
$$
d(p,q) = \sum_{i=1}^{n}{|p_i - q_i|}
$$

In [4]:
def manhattan_0(p, q):
    total = 0
    for i in range(len(p)):
        total += abs(p[i] - q[i])
    return total

def manhattan_1(p, q):
    p, q = np.array(p), np.array(q)
    return np.sum(np.abs(p - q))

def manhattan_2(X, q):
    if X.ndim > 2:
        raise ValueError("X has more than 2 dimensions.")
    m, n = X.shape
    q = q.reshape(1, -1)
    Q = np.repeat(q, m, axis=0)
    dist = np.sum(np.abs(X - Q), axis=1)
    return dist.reshape(-1, 1)

def manhattan_3(X, q):
    """
    Compute the Manhattan distance between each row of a 2-D array and a 1-D point.
    
    Parameters:
    X : 2-D array of shape (m, n)
    q : 1-D array of shape (n,)
    
    Returns:
    dist : 1-D array of shape (m,)
        The Manhattan distance between each row of X and q.
    """
    if X.ndim > 2:
        raise ValueError("X has more than 2 dimensions.")
    return np.sum(np.abs(X - q), axis=1)

In [5]:
# Test manhattan_0
print("Testing manhattan_0...")
%timeit -n 100 [manhattan_0(x, q) for x in X]

# Test manhattan_1
print("\nTesting manhattan_1...")
%timeit -n 100 [manhattan_1(x, q) for x in X]

# Test manhattan_2
print("\nTesting manhattan_2...")
%timeit -n 100 manhattan_2(X, q)

# Test manhattan_3
print("\nTesting manhattan_3...")
%timeit -n 100 manhattan_3(X, q)

Testing manhattan_0...
21 ms ± 324 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Testing manhattan_1...
7.82 ms ± 597 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Testing manhattan_2...
110 µs ± 8.69 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Testing manhattan_3...
113 µs ± 13.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Minkowski:
$$
d(p,q) = (\sum_{i=1}^{n}{(q_i - p_i)^n})^\frac{1}{n}
$$

In [10]:
def minkowski_0(p, q, n):
    total = 0.0
    for i in range(len(p)):
        total += (p[i] - q[i]) ** n
    return total ** (1 / n)

def minkowski_1(p, q, n):
    p, q = np.array(p), np.array(q)
    return np.power(np.sum(np.power(p - q, n)), 1/n)

def minkowski_2(X, q, n):
    if X.ndim > 2:
        raise ValueError("X has more than 2 dimensions.")
    m, _ = X.shape
    q = q.reshape(1, -1)
    Q = np.repeat(q, m, axis=0)
    dist = np.sum(np.power(X - Q, n), axis=1)
    return np.power(dist, 1/n).reshape(-1, 1)

def minkowski_3(X, q, n):
    """
    Compute the Minkowski distance between each row of a 2-D array and a 1-D point.
    
    Parameters:
    X : 2-D array of shape (m, n)
    q : 1-D array of shape (n,)
    n : Minkowski_n
    
    Returns:
    dist : 1-D array of shape (m,)
        The Minkowski_n distance between each row of X and q.
    """
    if X.ndim > 2:
        raise ValueError("X has more than 2 dimensions.")
    return np.power(np.sum(np.power((X - q), n), axis=1), 1/n)

In [11]:
n = 5
# Test minkowski_0
print("Testing minkowski_0...")
%timeit -n 100 [minkowski_0(x, q, n) for x in X]

# Test minkowski_1
print("\nTesting minkowski_1...")
%timeit -n 100 [minkowski_1(x, q, n) for x in X]

# Test minkowski_2
print("\nTesting minkowski_2...")
%timeit -n 100 minkowski_2(X, q, n)

# Test minkowski_3
print("\nTesting minkowski_3...")
%timeit -n 100 minkowski_3(X, q, n)

Testing minkowski_0...


C:\Users\moham\AppData\Local\Temp\ipykernel_16356\3387198962.py:5: RuntimeWarning: invalid value encountered in scalar power
  return total ** (1 / n)


23.8 ms ± 442 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Testing minkowski_1...


C:\Users\moham\AppData\Local\Temp\ipykernel_16356\3387198962.py:9: RuntimeWarning: invalid value encountered in power
  return np.power(np.sum(np.power(p - q, n)), 1/n)


16.4 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

Testing minkowski_2...


C:\Users\moham\AppData\Local\Temp\ipykernel_16356\3387198962.py:18: RuntimeWarning: invalid value encountered in power
  return np.power(dist, 1/n).reshape(-1, 1)


2.54 ms ± 311 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Testing minkowski_3...


C:\Users\moham\AppData\Local\Temp\ipykernel_16356\3387198962.py:35: RuntimeWarning: invalid value encountered in power
  return np.power(np.sum(np.power((X - q), n), axis=1), 1/n)


2.41 ms ± 164 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Getting the top K neighbours

In [34]:
k = 3 # for KNN
idx = 18
sample = X_test_normalized[idx]


distance_sample = euclidean_3(X_train_normalized, sample)
_idx = np.argsort(-distance_sample)
y_train_sorted = y_train[_idx]
y_train_neighbours = y_train_sorted[:k]

unique_labels, counts = np.unique(y_train_neighbours, return_counts=True)
unique_labels, counts

(array([0, 1]), array([2, 1], dtype=int64))

In [35]:
most_common_label = unique_labels[np.argmax(counts)]
most_common_label, y_test[idx]

(0, 1)

In [ ]:
class MyKNN():
    def __init__(self, X_train, y_train, sample, k=3):
        self.k = k
        self.X_train = X_train
        self.y_train = y_train
        
    def get_distance(self, X, sample, type='euclidean'):
        pass
        
    def _euclidean(self, X, sample):
        pass
        
    def _minkowski(self, X, sample, n):
        """
        Compute the Minkowski distance between each row of a 2-D array and a 1-D point.
        
        Parameters:
        X : 2-D array of shape (m, n)
        q : 1-D array of shape (n,)
        n : Minkowski_n
        
        Returns:
        dist : 1-D array of shape (m,)
            The Minkowski_n distance between each row of X and q.
        """
        if X.ndim > 2:
            raise ValueError("X has more than 2 dimensions.")
        return np.power(
            np.sum(
                np.power(
                    (X - q), n), axis=1
            ), 1/n
        )
        
    def _manhattan(self, X, sample):
        """
        Compute the Manhattan distance between each row of a 2-D array and a 1-D point.
        
        Parameters:
        X : 2-D array of shape (m, n)
        q : 1-D array of shape (n,)
        
        Returns:
        dist : 1-D array of shape (m,)
            The Manhattan distance between each row of X and q.
        """
        if X.ndim > 2:
            raise ValueError("X has more than 2 dimensions.")
        return np.sum(np.abs(X - sample), axis=1)

    def fit(self, sample):
        